### 資料參考來源：
https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html

In [6]:
import pandas as pd

url = "https://raw.github.com/pandas-dev/pandas/master/pandas/tests/io/data/csv/tips.csv"
tips = pd.read_csv(url)
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


##### 大部分的pandas 都會copy Series/Dataframe

In [21]:
# 排序
tips.sort_values("tip")                             #由小到大
tips.sort_values("tip", ascending=False)            #由大到小

#GroupBy 
tips.groupby("sex").size()                          #size() 只有取得這個欄位
tips.groupby("sex").count()                         #全部欄位的統計
tips.groupby("sex")["total_bill"].count()                       #針對total_bill欄位的統計

import numpy as np
# 加入npmpy數學統計套件
# SELECT day, AVG(tip), COUNT(*) FROM tips GROUP BY day;
tips.groupby("day").agg({"tip": np.mean, "day": np.size})

# SELECT smoker, day, COUNT(*), AVG(tip) FROM tips GROUP BY smoker, day;
tips.groupby(["smoker", "day"]).agg({"tip": [np.size, np.mean]})

,tip,day
day,,
Fri,2.734737,19
Sat,2.993103,87
Sun,3.255132,76
Thur,2.771452,62
